In [ ]:
%pip install tensorflow opencv-python matplotlib

In [2]:
# Standard dependencies
import cv2
import os
import random
import numpy as np # For re-shaping arrays
from matplotlib import pyplot as plt # Visualise images

# Tensorflow dependencies
import tensorflow as tf
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten

In [ ]:
# To avoid running out of memory, we restrict the GPU memory growth aka
# how many resources the model is consuming at any given time
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    # Just in case the device we are using has more than one gpu, we are 
    # making sure to restrict the usage of ALL of them
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
# Create files that will hold the anchor, positive and negative images:
# Anchor: The image we imput
# Positive: Images that match the anchor
# Negative: Images that are different from the anchor
anc_path = os.path.join("data","anchor")
pos_path = os.path.join("data","positive")
neg_path = os.path.join("data","negative")

os.makedirs(anc_path)
os.makedirs(pos_path)
os.makedirs(neg_path)


In [ ]:
# Uncompress the tar file that contains the images in the Wild Dataface
!tar -xf lfw.tgz

In [ ]:
# We take the images that we downloaded and place in the negative folder
# (these images will be used so that the machine can understand that the
# person whose image we are providing isn't the same as any of the ones in
# the negative folder)

# Go through all the directories in the lfw folder
for directory in os.listdir('lfw'):
    # Find all the images in said directory
    for file in os.listdir(os.path.join('lfw',directory)):
        # Replace the path of that image with the path of the negative folder
        # (aka place the image in the negative folder)
        previous_path = os.path.join('lfw', directory, file)
        new_path = os.path.join(neg_path, file)
        os.replace(previous_path, new_path)

In [3]:
# Importing this so that the images we save  all have different names
import uuid

# Now we will get the images we will require for the anchor and positive files

# Connect to the webcam
capture = cv2.VideoCapture(0) # Keep in mind this number might vary slightly
                              # so try out a few other numbers like 1, 2, 3, 4, 5 etc in case there is a problem
while (capture.isOpened()):
    return_value, frame = capture.read()

    # Keeping in mind that the images in the negative folder have a resolution of 250x250
    # we need out frames (aka the images we will capture) to be 250x250 as well
    frame = frame[120:370, 200:450, :]
    print(frame[2])

    # Show the camera feed
    cv2.imshow("Images", frame)

    # Add image to anchor if 'a' is pressed
    if (cv2.waitKey(1) & 0XFF == ord('a')):
        # Create the unique name and save the image
        name = os.path.join(anc_path, '{}.jpg'.format(uuid.uuid1()))
        cv2.imwrite(name, frame)
    
    # Add image to positive if 'p' is pressed
    if (cv2.waitKey(1) & 0XFF == ord('p')):
        # Create the unique name and save the image
        name = os.path.join(pos_path, '{}.jpg'.format(uuid.uuid1()))
        cv2.imwrite(name, frame)

    # Break by pressing he 'q' key
    if (cv2.waitKey(1) & 0XFF == ord('q')):
        break

# Release webcam
capture.release()
# Close the camera feed window
cv2.destroyAllWindows()

[[62 61 47]
 [65 62 48]
 [63 60 46]
 [62 59 48]
 [62 60 51]
 [61 59 50]
 [62 60 51]
 [62 59 50]
 [63 59 50]
 [62 59 48]
 [62 61 47]
 [66 60 49]
 [69 59 51]
 [66 60 50]
 [61 60 46]
 [61 59 44]
 [63 60 45]
 [65 59 43]
 [66 59 43]
 [66 60 46]
 [63 60 46]
 [66 60 46]
 [69 61 46]
 [66 60 44]
 [63 60 45]
 [63 60 46]
 [62 58 45]
 [62 59 44]
 [62 59 43]
 [63 59 47]
 [63 59 50]
 [62 57 49]
 [61 56 48]
 [60 56 45]
 [62 58 45]
 [61 59 45]
 [61 60 46]
 [64 61 47]
 [65 61 47]
 [64 61 47]
 [61 60 46]
 [62 60 49]
 [62 60 51]
 [62 60 49]
 [62 61 47]
 [62 61 49]
 [62 60 51]
 [62 60 51]
 [62 60 51]
 [64 60 51]
 [63 59 50]
 [61 58 47]
 [60 59 45]
 [59 57 46]
 [60 58 49]
 [61 59 50]
 [61 59 50]
 [61 59 50]
 [61 59 50]
 [60 58 47]
 [61 60 46]
 [61 60 46]
 [62 61 47]
 [62 61 47]
 [62 61 47]
 [64 61 47]
 [63 60 46]
 [61 59 45]
 [61 60 46]
 [61 59 45]
 [62 58 45]
 [64 60 46]
 [63 60 46]
 [66 62 49]
 [66 62 49]
 [65 61 47]
 [63 60 46]
 [66 62 49]
 [66 62 49]
 [66 62 49]
 [66 62 49]
 [66 62 49]
 [65 61 47]
 [65

In [ ]:
# Get 300 image paths from each image set
anchor = tf.data.Dataset.list_files(anc_path+'\*.jpg').take(300)
positive = tf.data.Dataset.list_files(pos_path+'\*.jpg').take(300)
negative = tf.data.Dataset.list_files(neg_path+'\*.jpg').take(300)

In [ ]:
# Scale and resize the images
def preprocess(file_path):
    # Get byte code of image (the file path) and then decode it
    byte_img = tf.io.read_file(file_path)
    img = tf.io.decode_jpeg(byte_img)

    img = tf.image.resize(img, (105, 105)) # Resizing out image according to the "Siamese Neural Networks"
                                           # research paper
    img = img / 255 # Scale every pixel value to 0-1 => scale the image
    return img

In [ ]:
# Depending on the inputs (anchor, positive) or (anchor, negative) we will be
# getting a result ( a label ) as follows:
# (anchor, positive) => 1
# (anchor, negative) => 0

positives = tf.data.Dataset.zip((anchor, positive, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
negatives = tf.data.Dataset.zip((anchor, negative, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))
data = positives.concatenate(negatives) # Combine the positives and negatives

In [ ]:
samples = data.as_numpy_iterator()

In [ ]:
example = samples.next()


In [ ]:
# Create function to scale and resize both images we pass
def twin_preprocess(anc, verification_image, label):
    return (preprocess(anc), preprocess(verification_image), label)

res = twin_preprocess(*example)
plt.imshow(res[0])

In [ ]:
# Dataloader pipeline
data = data.map(twin_preprocess) # ERROR TO_DO
data = data.cache()
data = data.shuffle(buffer_size=1024) # Simply mix the positive and negative images

In [ ]:
# Training partition
train_data = data.take((round(len(data)*0.7))) # Get 70% of the samples
train_data = train_data.batch(16) # Pass 16 images each time
train_data = train_data.prefetch(8) # Preprocess the next image beforehand


In [ ]:
train_sumples = train_data.as_numpy_iterator()

len(train_sumples.next()[0])

In [ ]:
# Testing Partition
test_data = data.skip((round(len(data)*0.7)))
test_data = test_data.take((round(len(data)*0.3)))
test_data = test_data.batch(16)
test_data = test_data.prefetch(8)

In [22]:
# Embedding layer

def make_embedding():
    # Input
    inp = Input(shape=(105, 105, 3), name="input_image")

    # First block
    # Convolusion layer
    c1 = Conv2D(64, (10, 10), activation="relu")(inp)
    # Max pooling layer
    m1  = MaxPooling2D(64, (2, 2), padding="same")(c1)

    # Second block
    c2 = Conv2D(128, (7, 7), activation="relu")(m1)
    m2 = MaxPooling2D(64, (2, 2), padding="same")(c2)

    # Third block
    c3 = Conv2D(128, (4, 4), activation="relu")(m2)
    m3 = MaxPooling2D(64, (2, 2), padding="same")(c3)

    # Final embedding block
    c4 = Conv2D(256, (4, 4), activation="relu")(m3)
    f1 = Flatten()(c4)
    d1 = Dense(4096, activation="sigmoid")(f1)

    return Model(inputs=[inp], outputs=[d1], name="embedding")

In [ ]:
inp = Input(shape=(105, 105, 3), name="input_image")
c1 = Conv2D(64, (10, 10), activation="relu")(inp)
m1  = MaxPooling2D(64, (2, 2), padding="same")(c1)
c2 = Conv2D(128, (7, 7), activation="relu")(m1)
m2 = MaxPooling2D(64, (2, 2), padding="same")(c2)
c3 = Conv2D(128, (4, 4), activation="relu")(m2)
m3 = MaxPooling2D(64, (2, 2), padding="same")(c3)
c4 = Conv2D(256, (4, 4), activation="relu")(m3)
f1 = Flatten()(c4) # will output 6*6*256
d1 = Dense(4096, activation="sigmoid")(f1)
mod = Model(inputs=[inp], outputs=[d1], name="embedding")


In [23]:
embedding = make_embedding()

In [25]:
embedding.summary()

Model: "embedding"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_image (InputLayer)     [(None, 105, 105, 3)]     0         
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 96, 96, 64)        19264     
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 48, 48, 64)        0         
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 42, 42, 128)       401536    
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 21, 21, 128)       0         
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 18, 18, 128)       262272    
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 9, 9, 128)         0 

In [19]:
# Distance layer
class L1Dist(Layer):
    def __init__(self, **kwargs):
        super().__init__()
    
    # We input our anchor embedding and either a posivite or negative embedding and output their distance
    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)

In [29]:
# Anchor input image
input_image = Input(name="input_img", shape=(105, 105, 3))
# Positive / Negative input image
validation_image = Input(name="validation_img", shape=(105, 105, 3))

inp_emb = embedding(input_image)
val_emb = embedding(validation_image)

siam = L1Dist()
distances = siam(inp_emb, val_emb)

# Classification layer
classifier = Dense(1, activation="sigmoid")(distances)

s = Model(inputs=[input_image, validation_image], outputs=classifier, name="Siamese Network")

s.summary()


Model: "Siamese Network"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_img (InputLayer)          [(None, 105, 105, 3) 0                                            
__________________________________________________________________________________________________
validation_img (InputLayer)     [(None, 105, 105, 3) 0                                            
__________________________________________________________________________________________________
embedding (Functional)          (None, 4096)         38960448    input_img[0][0]                  
                                                                 validation_img[0][0]             
__________________________________________________________________________________________________
l1_dist_4 (L1Dist)              (None, 4096)         0           embedding[6][0]    

In [ ]:
# Siamese model
def make_siamese_model():
    # Anchor input image
    input_image = Input(name="input_img", shape=(105, 105, 3))
    # Positive / Negative input image
    validation_image = Input(name="validation_img", shape=(105, 105, 3))

    # Combine Siamese distance
    siamese_layer = L1Dist()
    siamese_layer._name = "distance"
    distances = siamese_layer(embedding(input_image), embedding(validation_image))

    # Classification layer
    classifier = Dense(1, activation="sigmoid")(distances)

    return Model(inputs=[input_image, validation_image], outputs=classifier, name="Siamese Network")